参考: [[Python] スレッドで実装する](https://qiita.com/tchnkmr/items/b05f321fa315bbce4f77)

### スレッド(threading)

multiprocessingがコアを複数使った処理なのに対し，Threadingはメモリをたくさん使う

In [1]:
import time
import threading

def boil_udon():
    print('  うどんを茹でます。')
    time.sleep(3)
    print('  うどんが茹であがりました。')

def make_tuyu():
    print('  ツユをつくります。')
    time.sleep(2)
    print('  ツユができました。')

print('うどんを作ります。')

thread1 = threading.Thread(target=boil_udon)
thread2 = threading.Thread(target=make_tuyu)

thread1.start()
thread2.start()

thread1.join() # thread1が終わるまで待つ
thread2.join() # thread2が終わるまで待つ

print('盛り付けます。')
print('うどんができました。')

うどんを作ります。
  うどんを茹でます。
  ツユをつくります。
  ツユができました。
  うどんが茹であがりました。
盛り付けます。
うどんができました。


### スレッドプール(ThreadPoolExecutor)
一つのコアであまりにもたくさんのスレッドを立ててしまうと，かえって処理速度が落ちたり，メモリを無駄に食ったりする．  
  
そこで，一度に立てられるスレッドの数を制限する

In [2]:
from concurrent.futures import ThreadPoolExecutor
import time

def boil_udon():
    print('  うどんを茹でます。')
    time.sleep(3)
    print('  うどんが茹であがりました。')

# 1度にゆでられるうどんは3玉
tpe = ThreadPoolExecutor(max_workers=3)

print('うどんを10個茹でます。')
for _ in range(10):
    tpe.submit(boil_udon)

# tpeに登録した処理が全て終わるまで待つ
# 手が空いたら新しいうどんを作るので，最終的に全ての作業を終えられる
tpe.shutdown()

print('うどんが10個茹で上がりました。')

うどんを10個茹でます。
  うどんを茹でます。
  うどんを茹でます。
  うどんを茹でます。
  うどんが茹であがりました。
  うどんを茹でます。
  うどんが茹であがりました。  うどんが茹であがりました。
  うどんを茹でます。

  うどんを茹でます。
  うどんが茹であがりました。
  うどんを茹でます。
  うどんが茹であがりました。  うどんが茹であがりました。
  うどんを茹でます。

  うどんを茹でます。
  うどんが茹であがりました。
  うどんを茹でます。
  うどんが茹であがりました。
  うどんが茹であがりました。
  うどんが茹であがりました。
うどんが10個茹で上がりました。


### Future
submitメソッドの戻り値はFutureである  
Futureのresultメソッドを呼び出すと
- 処理がおわっている → 戻り値を取得
- 処理が終わっていない → 終わるまで待ち，戻り値を取得
    - 指定したtimeoutの時間を上回る → 例外を発生
  
他にもFutureでは
- running: スレッドが実行中ならTrue
- done: スレッドが終了していればTrue

In [3]:
from concurrent.futures import ThreadPoolExecutor
import time

def make_udon(kind):
    print('  %sうどんを作ります。' % kind)
    time.sleep(3)
    return kind + 'うどん'

kinds = ['たぬき', 'かけ', 'ざる', 'きつね', '天ぷら', '肉']
executor = ThreadPoolExecutor(max_workers=3)
futures = []

for kind in kinds:
    print('%sうどん オーダー入りました。' % kind)
    future = executor.submit(make_udon, kind)
    futures.append(future)

for future in futures:
    print('%sお待たせしました。' % future.result())

executor.shutdown()

たぬきうどん オーダー入りました。
  たぬきうどんを作ります。かけうどん オーダー入りました。

  かけうどんを作ります。
ざるうどん オーダー入りました。
  ざるうどんを作ります。
きつねうどん オーダー入りました。
天ぷらうどん オーダー入りました。
肉うどん オーダー入りました。
  きつねうどんを作ります。  天ぷらうどんを作ります。  肉うどんを作ります。たぬきうどんお待たせしました。



かけうどんお待たせしました。
ざるうどんお待たせしました。
きつねうどんお待たせしました。
天ぷらうどんお待たせしました。
肉うどんお待たせしました。


以上のような並列処理は
- 実装は簡単
- デバッグが困難
- 最適なスレッド数は見極める必要がある
  
という特徴があるので注意する